# 3. Wrapping a model into a class - the Single Particle Model

In the previous notebooks we learnt how to build models using PyBaMM. We implemented them directly in a notebook, which is very useful when building the model for the first time, but not so much if we want to use the model often or share it with other people. In these cases, it is better to wrap the model into a class. 

A [Python class](https://www.w3schools.com/python/python_classes.asp) is a "blueprint" to create an object. A Python object is a special variable that contain data (variables) and methods (functions) that can manipulate this data. In our case, the object will be a model, which we can then solve.

Building on our previous models, we will now implement the Single Particle Model for a full cell.

> WRITE SPM equations

## Setting up the model

- Explain the init and where the model is defined
- Explain additional attributes